In [ ]:
#pip install pymongo

In [1]:
# Import all required library for load data to mongo db
import sys
import pandas as pd
import pymongo
import json
import os
from pathlib import Path
from config import cloudM,cloudMpassword

In [2]:
# Declare variable to keep Database & connection name along with source flat file name
DatabaesName = 'CRYPTO'
collection_name = 'CRYPTO_DATA_FEED'
filepath = r"Data\bitstampUSD.csv" 
# Creating MongoDB connection object
mng_client = pymongo.MongoClient("mongodb+srv://"+cloudM + ":" + cloudMpassword + "@cluster0.cep5x.mongodb.net/myFirstDatabase?retryWrites=true&w=majority")
#create Database object
mng_db = mng_client[DatabaesName] 
#Create Collection object with-in Database
db_cm = mng_db[collection_name]
#define parent folder path 
cdir = Path().resolve()
#Define csv file read location 
file_res = os.path.join(cdir, filepath)
#print(file_res)

In [3]:
# Read CSV file and keep it in Panda's dataframe
rawdata_df = pd.read_csv(file_res)

# Convert Timestamp into TimeZone (ET)
rawdata_df['Timestamp'] = (pd.to_datetime(rawdata_df['Timestamp'], unit='s'))

#Set Index for time seriens 
rawdata_df = rawdata_df.set_index('Timestamp', verify_integrity=False)

In [4]:
# Rename Timestamp and Weighted_Price
##rawdata_df.rename(columns = {'Timestamp' : 'Date_Time', 'Volume_(BTC)':'Volume_BTC', 'Volume_(Currency)':'Volume_Currency'}, inplace = True)

# First thing is to fix the data for bars/candles where there are no trades. 
# Volume/trades are a single event so fill na's with zeroes for relevant fields...
rawdata_df['Volume_(BTC)'].fillna(value=0, inplace=True)
rawdata_df['Volume_(Currency)'].fillna(value=0, inplace=True)
rawdata_df['Weighted_Price'].fillna(value=0, inplace=True)

# next we need to fix the OHLC (open high low close) data which is a continuous timeseries so
# lets fill forwards those values...
rawdata_df['Open'].fillna(method='ffill', inplace=True)
rawdata_df['High'].fillna(method='ffill', inplace=True)
rawdata_df['Low'].fillna(method='ffill', inplace=True)
rawdata_df['Close'].fillna(method='ffill', inplace=True)


# check how we are looking now, should be nice and clean...
rawdata_df

,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
Timestamp,,,,,,,
2011-12-31 07:52:00,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
2011-12-31 07:53:00,4.39,4.39,4.39,4.39,0.000000,0.000000,0.000000
2011-12-31 07:54:00,4.39,4.39,4.39,4.39,0.000000,0.000000,0.000000
2011-12-31 07:55:00,4.39,4.39,4.39,4.39,0.000000,0.000000,0.000000
2011-12-31 07:56:00,4.39,4.39,4.39,4.39,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...
2021-03-30 23:56:00,58714.31,58714.31,58686.00,58686.00,1.384487,81259.372187,58692.753339
2021-03-30 23:57:00,58683.97,58693.43,58683.97,58685.81,7.294848,428158.146640,58693.226508
2021-03-30 23:58:00,58693.43,58723.84,58693.43,58723.84,1.705682,100117.070370,58696.198496


In [5]:
# Take convert complete dataset to hourly from seconds 
final_data = rawdata_df.resample('H').mean()

In [6]:
# remove index from final dataframe to insert to mongodb
final_data = final_data.reset_index()

In [7]:
# Transform dataframe to data disctionary to insert to MongoDB

#data_json = json.loads(final_data.to_json(orient='records'))
#db_cm.insert_many(data_json)
data_dict = final_data.to_dict(orient='records')

In [8]:
# Clean-up Collection before insert
x = db_cm.delete_many({})

#Insert dictionary object to collection in mongoDB
db_cm.insert_many(data_dict)

In [9]:
# Read data from Mongo DB to Pandas DataFrame
df = pd.DataFrame(list(db_cm.find()))

In [10]:
df.drop("_id", axis=1, inplace=True)
#df = df.set_index('Timestamp', verify_integrity=False)
df

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,2011-12-31 07:00:00,4.390000,4.390000,4.390000,4.390000,0.056948,0.250000,0.548750
1,2011-12-31 08:00:00,4.390000,4.390000,4.390000,4.390000,0.000000,0.000000,0.000000
2,2011-12-31 09:00:00,4.390000,4.390000,4.390000,4.390000,0.000000,0.000000,0.000000
3,2011-12-31 10:00:00,4.390000,4.390000,4.390000,4.390000,0.000000,0.000000,0.000000
4,2011-12-31 11:00:00,4.390000,4.390000,4.390000,4.390000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...
81061,2021-03-30 20:00:00,58686.619500,58717.499833,58649.440333,58678.720333,1.976271,116009.666363,58682.385127
81062,2021-03-30 21:00:00,58703.792500,58718.812833,58688.192000,58709.001000,0.960084,56335.721540,58705.094588
81063,2021-03-30 22:00:00,58788.743667,58806.029333,58770.925667,58788.207333,1.133653,66663.661684,58788.454429
81064,2021-03-30 23:00:00,58666.797500,58680.546833,58653.793833,58668.532333,1.305488,76565.752741,56711.191789
